# Librerias

In [1]:
import pandas as pd
import numpy as np
"""graficas"""
import matplotlib.pyplot as plt
import seaborn as sns
""""codificar datos categoricos"""
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer

# **Modificando**

# Importacion de datos

In [2]:
"""importando flujos limpios"""
df_normal = pd.read_csv('../../araData/Z/normal_1.7sincategorizar.csv', low_memory=False)
df_normal.shape

(94285, 52)

In [3]:
df_normal.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 94285 entries, 0 to 94284
Data columns (total 52 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   RunTime     94285 non-null  float64
 1   Dur         94285 non-null  float64
 2   SrcDur      94285 non-null  float64
 3   DstDur      94285 non-null  float64
 4   Mean        94285 non-null  float64
 5   StdDev      94285 non-null  float64
 6   Sum         94285 non-null  float64
 7   Min         94285 non-null  float64
 8   Max         94285 non-null  float64
 9   Dir         94285 non-null  object 
 10  Proto       94285 non-null  object 
 11  Dport       94285 non-null  object 
 12  Cause       94285 non-null  object 
 13  TotPkts     94285 non-null  int64  
 14  SrcPkts     94285 non-null  int64  
 15  DstPkts     94285 non-null  int64  
 16  TotBytes    94285 non-null  int64  
 17  SrcBytes    94285 non-null  int64  
 18  DstBytes    94285 non-null  int64  
 19  TotAppByte  94285 non-nul

In [4]:
df_normal.Proto.value_counts()

udp    52333
tcp    41952
Name: Proto, dtype: int64

In [5]:
df_normal.Dport.value_counts()

domain         50976
https          34716
6653            4962
http            2137
1900             603
hostmon          349
netbios-ns       225
mdns             200
netbios-dgm       82
ssh               17
22790              8
8612               2
8610               2
53592              1
57706              1
47008              1
telnet             1
53542              1
53576              1
Name: Dport, dtype: int64

### Elimino caracteristicas por decision (repeticion, innecesarias)

In [6]:
"""eliminacion de variables de categoricas, continuas"""
df_normal.drop(columns=['RunTime','Mean','StdDev','Sum','Min','Max','Dir','Cause','State','Retrans','SrcRetra','DstRetra',
'pRetran','pSrcRetr','pDstRetr'], axis=1, inplace=True)

In [7]:
df_normal.shape

(94285, 37)

## Codificacion de Variables categóricas

### Reemplazar datos de texto a datos numericos (Puertos)

In [8]:
df_normal.Dport.replace({'domain':53,'http':80, 'https':443,'hostmon': 5355,'netbios-ns':137,'mdns':5353,'netbios-dgm':138,
'ssh':22,'telnet':23},inplace=True )

In [9]:
df_normal.Dport.value_counts()

53       50976
443      34716
6653      4962
80        2137
1900       603
5355       349
137        225
5353       200
138         82
22          17
22790        8
8612         2
8610         2
53592        1
57706        1
47008        1
23           1
53542        1
53576        1
Name: Dport, dtype: int64

### Reemplazar datos de texto a datos numericos (Protocolos)

In [10]:
df_normal.Proto.replace({'tcp':6,'udp':17},inplace=True )

In [11]:
df_normal.Proto.value_counts()

17    52333
6     41952
Name: Proto, dtype: int64

In [12]:
df_normal['Dport'] = df_normal['Dport'].astype('int32')
df_normal['Proto'] = df_normal['Proto'].astype('int32')

### Discretizar Puertos (Conocidos-Registrados-Dinamicos)

In [13]:
binning_ports= np.array(df_normal['Dport'])
binning_ports

array([ 443,   80,   80, ..., 6653, 6653, 6653])

In [14]:
"""binning de puertos"""
binning_ports = pd.cut(binning_ports, bins=[0, 1023, 49151, 65535], 
                    labels=["aportSystem","bportUser","cportDynamic"],
                    include_lowest=True,
                    retbins=True
                )

### Crear DF de puertos y protocoos 

In [15]:
"""creo un df con el binning de puertos"""
df_ports = pd.DataFrame({'dstPort':np.array(binning_ports[0])})

In [16]:
"""creo un df obteniendo los protocolos"""
df_protos = pd.DataFrame({'Protoc':np.array(df_normal['Proto'])})

# no hago

### Codificar Label como Ordinal (temporal)

In [17]:
labelencoder_y = LabelEncoder()
"""codificar la variable objetivo"""
df_normal['Label'] = labelencoder_y.fit_transform(df_normal['Label'])

### Establecer variables categoricas nominales a Protocolos y ordinales a Puertos, ambas se trabajan con One Hot

### **Protocolos**

In [18]:
"""codificar protocolo"""
codificador= ColumnTransformer(
    [('one_hot_encoder', OneHotEncoder(categories='auto'), ['Protoc'])],
    remainder='passthrough'
    )
"""codificar puertos segun el rango"""
arr_protos = np.array(codificador.fit_transform(df_protos), dtype=int)

In [19]:
codificador.get_feature_names_out()

array(['one_hot_encoder__Protoc_6', 'one_hot_encoder__Protoc_17'],
      dtype=object)

In [20]:
df_protos = pd.DataFrame(arr_protos, columns= ['protoTcp', 'protoUdp'])

### **Puertos**

In [21]:
"""codificar puertos"""
codificadorports= ColumnTransformer(
    [('one_hot_encoder', OneHotEncoder(categories='auto'), ['dstPort'])],
    remainder='passthrough'
    )
"""codificar puertos segun el rango"""
arr_ports = np.array(codificadorports.fit_transform(df_ports), dtype=int)

In [22]:
codificadorports.get_feature_names_out()

array(['one_hot_encoder__dstPort_aportSystem',
       'one_hot_encoder__dstPort_bportUser',
       'one_hot_encoder__dstPort_cportDynamic'], dtype=object)

In [23]:
df_ports = pd.DataFrame(arr_ports, columns= ['portSystem','portUser','portDynamic'])

## Eliminar las columnas del df original que han sido tratadas como variables categoricas

In [24]:
"""eliminar variable categorica Dport"""
df_normal.drop(columns=['Dport','Proto'], axis=1, inplace=True)

In [25]:
df_normal.columns.values

array(['Dur', 'SrcDur', 'DstDur', 'TotPkts', 'SrcPkts', 'DstPkts',
       'TotBytes', 'SrcBytes', 'DstBytes', 'TotAppByte', 'SAppBytes',
       'DAppBytes', 'Load', 'SrcLoad', 'DstLoad', 'Loss', 'SrcLoss',
       'DstLoss', 'pLoss', 'pSrcLoss', 'pDstLoss', 'Rate', 'SrcRate',
       'DstRate', 'TcpRtt', 'SynAck', 'AckDat', 'Offset', 'sMeanPktSz',
       'dMeanPktSz', 'sMaxPktSz', 'dMaxPktSz', 'sMinPktSz', 'dMinPktSz',
       'Label'], dtype=object)

## Concatenar caracteristicas en un solo DF

In [26]:
df_traffic = pd.concat([df_protos, df_ports, df_normal], axis=1)

In [27]:
df_traffic.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 94285 entries, 0 to 94284
Data columns (total 40 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   protoTcp     94285 non-null  int32  
 1   protoUdp     94285 non-null  int32  
 2   portSystem   94285 non-null  int32  
 3   portUser     94285 non-null  int32  
 4   portDynamic  94285 non-null  int32  
 5   Dur          94285 non-null  float64
 6   SrcDur       94285 non-null  float64
 7   DstDur       94285 non-null  float64
 8   TotPkts      94285 non-null  int64  
 9   SrcPkts      94285 non-null  int64  
 10  DstPkts      94285 non-null  int64  
 11  TotBytes     94285 non-null  int64  
 12  SrcBytes     94285 non-null  int64  
 13  DstBytes     94285 non-null  int64  
 14  TotAppByte   94285 non-null  int64  
 15  SAppBytes    94285 non-null  int64  
 16  DAppBytes    94285 non-null  int64  
 17  Load         94285 non-null  float64
 18  SrcLoad      94285 non-null  float64
 19  DstL